In [ ]:
# Application of order reduction techniques to toy dataset prior to use with generative model
# Using toy dataset: sklearn cancer research
#
# Author: Alex Braafladt
#
# Version: v1 Initial creation 4/29/2022
#
# References:
#   -https://openturns.github.io/openturns/latest/theory/probabilistic_modeling/copulas.html#copula
#   -https://en.wikipedia.org/wiki/Copula_(probability_theory)